In [1]:
#Importing the required libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading the files

data_main = pd.read_csv('TRAIN.csv')
test_main = pd.read_csv('TEST_FINAL.csv')

In [3]:
data_main.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [4]:
data_main['#Order'].describe()

count    188340.000000
mean         68.205692
std          30.467415
min           0.000000
25%          48.000000
50%          63.000000
75%          82.000000
max         371.000000
Name: #Order, dtype: float64

In [5]:
#Removing the outliers as per mean + 2.5*std
data_main = data_main[data_main['#Order'] < 145]

In [6]:
data_main['Sales'].describe()

count    183649.000000
mean      41335.544699
std       15974.180278
min           0.000000
25%       30179.100000
50%       39192.000000
75%       50715.000000
max      137356.560000
Name: Sales, dtype: float64

In [7]:
41335.544699+2.5*15974.180278

81270.995394

In [8]:
#Removing the outliers as per mean + 2.5*std
data_main = data_main[data_main['Sales'] < 82000]

In [9]:
df_train= data_main.copy()
df_test = test_main.copy()

In [10]:
print(df_train.shape, df_test.shape)

(180843, 10) (22265, 8)


In [11]:
df_train.head(10)

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52
5,T1000006,249,S1,L3,R2,2018-01-01,1,Yes,39,34211.22
6,T1000007,248,S1,L1,R2,2018-01-01,1,Yes,40,35352.66
7,T1000008,247,S1,L1,R3,2018-01-01,1,Yes,64,52650.00
8,T1000009,246,S3,L1,R3,2018-01-01,1,Yes,62,42633.78
9,T1000010,254,S4,L1,R1,2018-01-01,1,Yes,87,62572.80


In [12]:
df_test.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
0,T1188341,171,S4,L2,R3,2019-06-01,0,No
1,T1188342,172,S1,L1,R1,2019-06-01,0,No
2,T1188343,173,S4,L2,R1,2019-06-01,0,No
3,T1188344,174,S1,L1,R4,2019-06-01,0,No
4,T1188345,170,S1,L1,R2,2019-06-01,0,No


In [13]:
df_train.dtypes

ID                object
Store_id           int64
Store_Type        object
Location_Type     object
Region_Code       object
Date              object
Holiday            int64
Discount          object
#Order             int64
Sales            float64
dtype: object

In [14]:
#Converting data to datetime format

df_train['Date'] = pd.to_datetime(df_train['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])

In [15]:
df_train.isnull().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Date             0
Holiday          0
Discount         0
#Order           0
Sales            0
dtype: int64

In [16]:
df_train.Date.dt.year.value_counts()

2018    128349
2019     52494
Name: Date, dtype: int64

In [17]:
df_test.Date.dt.year.value_counts()

2019    22265
Name: Date, dtype: int64

In [18]:
df_train.dropna(axis = 0, inplace = True)

In [19]:
df_train['Store_id'].value_counts()

38     516
174    516
213    516
141    516
269    516
      ... 
146    194
160    186
312    133
337    110
175     85
Name: Store_id, Length: 365, dtype: int64

In [20]:
df_train['Store_Type'].value_counts()

S1    88390
S4    39353
S2    28891
S3    24209
Name: Store_Type, dtype: int64

In [21]:
df_train['Region_Code'].value_counts()

R1    59182
R2    53013
R3    43314
R4    25334
Name: Region_Code, dtype: int64

In [22]:
df_train['Location_Type'].value_counts()

L1    84176
L2    42003
L3    29901
L5    13930
L4    10833
Name: Location_Type, dtype: int64

In [23]:
#Creating new features from Date column

df_train['weeknum'] = df_train.Date.dt.week
df_test['weeknum'] = df_test.Date.dt.week

df_train['Date_OM'] = df_train.Date.dt.day
df_test['Date_OM'] = df_test.Date.dt.day

df_train['Month'] = df_train.Date.dt.month
df_test['Month'] = df_test.Date.dt.month

df_train['Year'] = df_train.Date.dt.year
df_test['Year'] = df_test.Date.dt.year

In [24]:
#Renaming #Order to Order_Count to ease code writing
df_train.rename(columns={'#Order':'Order_Count'}, inplace = True)

In [25]:
df_train.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order_Count,Sales,weeknum,Date_OM,Month,Year
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84,1,1,1,2018
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12,1,1,1,2018
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20,1,1,1,2018
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16,1,1,1,2018
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52,1,1,1,2018


In [26]:
data = df_train.drop(['Order_Count', 'Sales'], axis = 1)
test = df_test.copy()

In [27]:
#Creating the rolling weeknumber feature

data['weeknum_rolling'] = data['Date'].apply(lambda x: x.week if x.year == 2018 else x.week+52)
test['weeknum_rolling'] = test['Date'].apply(lambda x: x.week if x.year == 2018 else x.week+52)

In [28]:
#Creating is_weekday feature

data['is_weekday'] = data['Date'].apply(lambda x: 1 if x.weekday() < 5 else 0)
test['is_weekday'] = test['Date'].apply(lambda x: 1 if x.weekday() < 5 else 0)

In [29]:
#Creating a function to determine the quarter of the year

def year_quarter(month):
    if month > 9:
        return 4
    elif month > 6:
        return 3
    elif month > 3:
        return 2
    else:
        return 1

In [30]:
data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,weeknum,Date_OM,Month,Year,weeknum_rolling,is_weekday
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,1,1,1,2018,1,1
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,1,1,1,2018,1,1
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,1,1,1,2018,1,1
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,1,1,1,2018,1,1
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,1,1,1,2018,1,1


In [31]:
#Dropping the columns that are not required

data.drop(['ID', 'Date'], axis = 1, inplace = True)
test.drop(['ID', 'Date'], axis = 1, inplace = True)

In [32]:
#Mapping discount 'Yes' to 1 and 'No' to 0

mapping_Discount = {'Yes':1, 'No': 0}
data['Discount'] = data['Discount'].map(mapping_Discount)
test['Discount'] = test['Discount'].map(mapping_Discount)

In [33]:
#Performing one hot encoding on Region_Code

data = pd.get_dummies(data, columns = ['Region_Code'])
test = pd.get_dummies(test, columns = ['Region_Code'])

In [34]:
#Creating an Ordinal Encoder for Store_Type and Location_Type since they signify type, not the code

from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

In [35]:
data['Store_Type_Enc'] = oe.fit_transform(data['Store_Type'].values.reshape(-1,1))
test['Store_Type_Enc'] = oe.transform(test['Store_Type'].values.reshape(-1,1))

data['Location_Type_Enc'] = oe.fit_transform(data['Location_Type'].values.reshape(-1,1))
test['Location_Type_Enc'] = oe.transform(test['Location_Type'].values.reshape(-1,1))

In [36]:
data.head()

,Store_id,Store_Type,Location_Type,Holiday,Discount,weeknum,Date_OM,Month,Year,weeknum_rolling,is_weekday,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,Store_Type_Enc,Location_Type_Enc
0,1,S1,L3,1,1,1,1,1,2018,1,1,1,0,0,0,0.0,2.0
1,253,S4,L2,1,1,1,1,1,2018,1,1,1,0,0,0,3.0,1.0
2,252,S3,L2,1,1,1,1,1,2018,1,1,1,0,0,0,2.0,1.0
3,251,S2,L3,1,1,1,1,1,2018,1,1,1,0,0,0,1.0,2.0
4,250,S2,L3,1,1,1,1,1,2018,1,1,0,0,0,1,1.0,2.0


In [37]:
data.drop(['Location_Type', 'Store_Type'], axis = 1, inplace = True)
test.drop(['Location_Type', 'Store_Type'], axis = 1, inplace = True)

In [38]:
#Importing the XGBRegressor

from xgboost import XGBRegressor

In [39]:
model = XGBRegressor(objective = 'reg:squarederror', n_estimators= 120, max_depth = 4)

In [40]:
#Creating the training data for fitting to predict Order_Count for test

X_train = data
y_train = df_train['Order_Count']

In [41]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=120,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [42]:
#predicting Order count for test data

order_preds = model.predict(test)

In [43]:
test['Order_Count'] = order_preds

In [44]:
X_train['Order_Count'] = df_train['Order_Count']
y_train = data_main['Sales']

In [45]:
#Creating a new model to predict Sales for test data

new_model = XGBRegressor(objective = 'reg:squarederror', n_estimators= 120, max_depth = 4)
new_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=120,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [46]:
preds_final = new_model.predict(test)

In [47]:
#Creating a dataframe for submissions

submission = pd.DataFrame()
submission['ID'] = test_main['ID']
submission['Sales'] = preds_final

In [48]:
submission.head()

,ID,Sales
0,T1188341,66988.140625
1,T1188342,47413.191406
2,T1188343,70383.914062
3,T1188344,49782.582031
4,T1188345,48906.929688


In [51]:
#Exporting submission
submission.to_csv('S25.csv', index = False)

In [52]:
preds_final.min()

19552.672